# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary classifier.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use a random forest classifier, as well as another classifier of your choice; either logistic regression, SVM, or KNN. 

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Set up a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Write 4 functions to extract these items (one function for each): location, company, job title, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```

##### - Make sure these functions are robust and can handle cases where the data/field may not be available.
>- Remember to check if a field is empty or None for attempting to call methods on it
>- Remember to use try/except if you anticipate errors.

- **Test** the functions on the results above and simple examples

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.
- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the l=New+York and the start=10. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).
##### Complete the following code to collect results from multiple cities and starting points.
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

### Save your results as a CSV

In [1]:
# Export to csv
import pandas as pd
import re
data=pd.read_csv('jobs_data2.csv').drop('Unnamed: 0', axis=1)
df=data.drop_duplicates().copy()
df=df.reset_index(drop=True)
df.head()

,location,company,job_title,salary,city
0,"Houston, TX",MD Anderson Cancer Center,Institute Research Scientist - Translational B...,"$95,000 - $142,000 a year",Houston
1,"Houston, TX",MD Anderson Cancer Center,Computational Scientist,"$76,400 - $114,600 a year",Houston
2,"Houston, TX",MD Anderson Cancer Center,Research Scientist - Neuro-Oncology - Research,"$51,600 - $77,400 a year",Houston
3,"Houston, TX",MD Anderson Cancer Center,Research Scientist - Thoracic H&N Med Oncology...,"$51,600 - $77,400 a year",Houston
4,"Houston, TX","Earl Carl Institute for Legal & Social Policy,...",Part Time Academic Research and Writing Policy...,"$20,000 - $25,000 a year",Houston


In [2]:
df['time_frame']=['yr' if ('year' in _) \
                  else 'mo' if ('month' in _) \
                  else 'hr' if ('hour' in _) \
                  else '-'
                  for _ in df['salary']]

In [3]:
df['salary']=df['salary'].str.replace(' a year', '')
df['salary']=df['salary'].str.replace(' a month', '')
df['salary']=df['salary'].str.replace(' an hour', '')
df['salary']=df['salary'].str.replace('$', '')
df['salary']=df['salary'].str.replace('-','')
df['salary']=df['salary'].str.replace(',','')
df['salary']=df['salary'].str.strip()

In [4]:
def stringparse(test):
    if re.match(r'^\d+\ +\d+', test):
        return (float(re.findall('\d+', test)[0])+float(re.findall('\d+', test)[1]))/2
    elif re.match('\d+\.\d+\ +\d+\.+\d+', test):
        return (float(re.findall('\d+\.\d+', test)[0])+float(re.findall('\d+\.\d+', test)[1]))/2
    else:
        return (float(test))

In [5]:
df['sal_avg']=df['salary'].apply(stringparse)
df['sal_yr']=[_[1]['sal_avg'] if _[1]['time_frame']=='yr' \
              else _[1]['sal_avg']*12 if _[1]['time_frame']=='mo' \
              else _[1]['sal_avg']*2000 if _[1]['time_frame']=='hr' \
              else 0 for _ in df.iterrows()]
df.head()

,location,company,job_title,salary,city,time_frame,sal_avg,sal_yr
0,"Houston, TX",MD Anderson Cancer Center,Institute Research Scientist - Translational B...,95000 142000,Houston,yr,118500.0,118500.0
1,"Houston, TX",MD Anderson Cancer Center,Computational Scientist,76400 114600,Houston,yr,95500.0,95500.0
2,"Houston, TX",MD Anderson Cancer Center,Research Scientist - Neuro-Oncology - Research,51600 77400,Houston,yr,64500.0,64500.0
3,"Houston, TX",MD Anderson Cancer Center,Research Scientist - Thoracic H&N Med Oncology...,51600 77400,Houston,yr,64500.0,64500.0
4,"Houston, TX","Earl Carl Institute for Legal & Social Policy,...",Part Time Academic Research and Writing Policy...,20000 25000,Houston,yr,22500.0,22500.0


city_list=['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+City']

## Predicting salaries using Random Forests + Another Classifier

#### Load in the the data of scraped salaries

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [6]:
df['above_med']=(df['sal_yr']>df['sal_yr'].median())

#### Thought experiment: What is the baseline accuracy for this model?

50/50

#### Create a Random Forest model to predict High/Low salary using Sklearn. Start by ONLY using the location as a feature. 

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [8]:
X=pd.get_dummies(df['city'])
y=df['above_med']
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=.2)
rforest=RandomForestClassifier(n_estimators=1000)
rforest.fit(X_train, y_train)
print 'RForest Acc., Location Only: '+ str(rforest.score(X_test, y_test))

RForest Acc., Location Only: 0.59375


#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title or whether 'Manager' is in the title. 
- Then build a new Random Forest with these features. Do they add any value?
- After creating these variables, use count-vectorizer to create features based on the words in the job titles.
- Build a new random forest model with location and these new features included.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
titles=df['job_title']
companies=df['company']

In [10]:
cvec_titles=CountVectorizer(stop_words='english', ngram_range=(2,2))
cvec_titles.fit(titles)
cvec_titles_data=cvec_titles.transform(titles)
df_titles=pd.DataFrame(cvec_titles_data.todense(), columns=cvec_titles.get_feature_names())

In [11]:
X=pd.concat([df_titles, X], axis=1)
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=.2)

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model, as well as any other metrics you feel are appropriate. 

In [12]:
grid_params={'n_estimators':[10,25,100,1000],
            'max_features':[15,25,50,250]}
grid=GridSearchCV(RandomForestClassifier(), param_grid=grid_params, cv=10, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 25, 100, 1000], 'max_features': [15, 25, 50, 250]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [13]:
rforest=grid.best_estimator_
rforest.fit(X_train, y_train)
print 'RForest Acc., Location+Description CVec: '+str(rforest.score(X_test, y_test))

RForest Acc., Location+Description CVec: 0.75


In [14]:
print grid.best_params_

{'max_features': 15, 'n_estimators': 1000}


#### Repeat the model-building process with a non-tree-based method.

In [15]:
from sklearn.neighbors import KNeighborsClassifier
grid_params={'p':[1,2,3],
            'weights':['uniform', 'distance'],
            'algorithm':['ball_tree', 'kd_tree', 'brute'],
            'n_neighbors':[2,5,10]}

In [16]:
grid=GridSearchCV(KNeighborsClassifier(n_jobs=-1), param_grid=grid_params)
grid.fit(X_train, y_train)
knn=grid.best_estimator_

In [17]:
print grid.best_params_
print 'knn-score'+str(knn.score(X_test, y_test))

{'n_neighbors': 2, 'weights': 'distance', 'algorithm': 'kd_tree', 'p': 1}
knn-score0.640625


In [18]:
X_train.shape

(256, 631)

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the job descriptions. 
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [45]:
feature_importances = pd.DataFrame(rforest.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
pd.DataFrame(feature_importances)

,importance
data scientist,0.087929
research analyst,0.030084
data analyst,0.029052
data engineer,0.023447
Portland,0.015948
San+Francisco,0.015810
machine learning,0.014592
Washington+City,0.013437
senior data,0.012736
research associate,0.009951


### TESTING

In [83]:
test_titles=['Data Scientist', 'Research Analyst', 'Data Engineer', 'Research Associate',
             'Machine Learning Engineer', 'Research Assistant', 'Statistical Analyst', 
             'Quantitative Analyst','Data Science Developer', 'Database Analyst',
             'Data Specialist', 'Research Analyst', 'Predictive Analyst', 'Machine Learning Specialist',
            'Database Engineer', 'Statistical Analyst', 'Software Engineer', 'Software Developer', 'Database Developer']
             
city_list=['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
           'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
           'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+City']
dftest_title_list=[]
dftest_city_list=[]
for tit in test_titles:
    for cit in city_list:
        dftest_title_list.append(tit)
        dftest_city_list.append(cit)
test_df=pd.DataFrame({'job_title':dftest_title_list, 'city':dftest_city_list})

In [84]:
test_X=pd.get_dummies(test_df['city'])
test_titlevecs=cvec_titles.transform(test_df['job_title'])
test_df_titles=pd.DataFrame(test_titlevecs.todense(), columns=cvec_titles.get_feature_names())
test_X=pd.concat([test_df_titles, test_X], axis=1)

In [85]:
y_pred=rforest.predict(test_X)

In [86]:
test_df['above_med']=y_pred

In [87]:
pd.set_option("display.max_rows",300)
test_df.sort_values('job_title')

,city,job_title,above_med
39,Atlanta,Data Engineer,True
34,San+Francisco,Data Engineer,True
35,Austin,Data Engineer,True
36,Seattle,Data Engineer,True
37,Los+Angeles,Data Engineer,True
38,Philadelphia,Data Engineer,True
40,Dallas,Data Engineer,True
41,Pittsburgh,Data Engineer,True
42,Portland,Data Engineer,True
43,Phoenix,Data Engineer,True


In [88]:
test_df.groupby('job_title')['above_med'].sum().sort_values(ascending=False)

job_title
Data Engineer                  16.0
Data Scientist                 16.0
Machine Learning Engineer      16.0
Quantitative Analyst           15.0
Machine Learning Specialist    15.0
Statistical Analyst            12.0
Data Science Developer         11.0
Software Engineer               6.0
Research Assistant              5.0
Predictive Analyst              5.0
Database Engineer               5.0
Database Developer              5.0
Database Analyst                5.0
Data Specialist                 5.0
Software Developer              5.0
Research Associate              1.0
Research Analyst                0.0
Name: above_med, dtype: float64

In [89]:
test_df.sort_values('city')

,city,job_title,above_med
151,Atlanta,Database Analyst,False
55,Atlanta,Research Associate,False
247,Atlanta,Statistical Analyst,True
263,Atlanta,Software Engineer,False
39,Atlanta,Data Engineer,True
71,Atlanta,Machine Learning Engineer,True
231,Atlanta,Database Engineer,False
87,Atlanta,Research Assistant,False
215,Atlanta,Machine Learning Specialist,True
279,Atlanta,Software Developer,False


In [91]:
test_df.groupby('city')['above_med'].sum().sort_values(ascending=False)

city
San+Francisco      17.0
Washington+City    16.0
Philadelphia       16.0
Los+Angeles        16.0
Austin             16.0
Atlanta             8.0
Chicago             7.0
Seattle             6.0
New+York            6.0
Miami               6.0
Denver              6.0
Pittsburgh          5.0
Phoenix             5.0
Houston             5.0
Dallas              5.0
Portland            3.0
Name: above_med, dtype: float64

In [54]:
df

,location,company,job_title,salary,city,time_frame,sal_avg,sal_yr,above_med
0,"Houston, TX",MD Anderson Cancer Center,Institute Research Scientist - Translational B...,95000 142000,Houston,yr,118500.000,118500.0,True
1,"Houston, TX",MD Anderson Cancer Center,Computational Scientist,76400 114600,Houston,yr,95500.000,95500.0,True
2,"Houston, TX",MD Anderson Cancer Center,Research Scientist - Neuro-Oncology - Research,51600 77400,Houston,yr,64500.000,64500.0,False
3,"Houston, TX",MD Anderson Cancer Center,Research Scientist - Thoracic H&N Med Oncology...,51600 77400,Houston,yr,64500.000,64500.0,False
4,"Houston, TX","Earl Carl Institute for Legal & Social Policy,...",Part Time Academic Research and Writing Policy...,20000 25000,Houston,yr,22500.000,22500.0,False
5,"Houston, TX",Rice University,Research Scientist,40800 79100,Houston,yr,59950.000,59950.0,False
6,"Houston, TX",Baylor College of Medicine,Quality Assurance Analyst (Research),43794,Houston,yr,43794.000,43794.0,False
7,"Houston, TX",MD Anderson Cancer Center,Research Admin Analyst,45500 82000,Houston,yr,63750.000,63750.0,False
8,"Houston, TX",MD Anderson Cancer Center,Research Statistical Analyst -Bioinformatics &...,66400 99600,Houston,yr,83000.000,83000.0,False
9,"Houston, TX",MD Anderson Cancer Center,Data Analyst- Immunology,42400 63600,Houston,yr,53000.000,53000.0,False
